In [ ]:
import pandas as pd
import nltk
import re

Loading Dataset

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',encoding="ISO-8859-1")

**Data Preprocessing**

Encoding the categorical coulmn

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data['v1']=le.fit_transform(data['v1'])

In [ ]:
data.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 

In [ ]:
pe=PorterStemmer()
arr=[]

In [ ]:
data.shape

(5572, 5)

In [ ]:
for i in range(5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()
  msg=[pe.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  arr.append(msg)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000)

In [ ]:
x=cv.fit_transform(arr).toarray()

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
x.shape

(5572, 6221)

In [ ]:
y=data['v1'].values

In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

Train and Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

**MOdel building**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model=Sequential()

In [ ]:
model.add(Dense(units=6221,activation='relu'))
model.add(Dense(units=10000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 64s 441ms/step - loss: 0.1266 - accuracy: 0.9670
Epoch 2/5
140/140 [==============================] - 59s 424ms/step - loss: 0.0069 - accuracy: 0.9984
Epoch 3/5
140/140 [==============================] - 59s 424ms/step - loss: 8.1224e-04 - accuracy: 0.9998
Epoch 4/5
140/140 [==============================] - 61s 433ms/step - loss: 1.1299e-04 - accuracy: 1.0000
Epoch 5/5
 25/140 [====>.........................] - ETA: 49s - loss: 4.9234e-05 - accuracy: 1.0000

Saving Model

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/IBM

/content/drive/MyDrive/IBM


In [ ]:
model.save('spamDetector.h5')

Testing Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model('/content/drive/MyDrive/IBM/spamDetector.h5')

In [ ]:
def preprocessing(text):
  text=re.sub('[^a-zA-Z]',' ',text)
  text=text.lower()
  text=text.split()
  text=[pe.stem(word) for word in text if not word in set(stopwords.words('english'))]
  text=' '.join(text)
  return text

In [ ]:
message='WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

In [ ]:
message=preprocessing(message)

In [ ]:
message

'winner valu network custom select receivea prize reward claim call claim code kl valid hour'

In [ ]:
result=model.predict(cv.transform([message]))

1/1 [==============================] - 0s 377ms/step


In [ ]:
result

array([[1.]], dtype=float32)

In [ ]:
if(result>0.5):
  print("Spam Message!!!")
else:
  print("Not Spam")

Spam Message!!!
